In [1]:
import sys
import re
from dataclasses import dataclass
from functools import reduce
from operator import mul

sys.path.append('../utils')
from pyutils import *

In [2]:
puzzle = readutf8('input.txt')
rmsize_puzzle = (101, 103)

In [3]:
sample = readutf8('sample.txt')
rmsize_sample = (11, 7)

In [28]:
@dataclass
class Robot:
    pos: Point
    vel: Point

class RobotRoom:
    ROBOT_REGEX = re.compile(r"[pv]=(\d+|-\d+),(\d+|-\d+)")
    def __init__(self, robot_string: list[str], w: int, h: int):
        self.robots: list[Robot] = []
        for line in robot_string.split('\n'):
            if matches := self.ROBOT_REGEX.findall(line):
                self.robots.append(Robot(*[tuple(map(int, g)) for g in matches]))
        self.w: int = w
        self.h: int = h
    
    def count_robots(self) -> dict[Point, int]:
        count: dict[Point, int] = {}
        for rob in self.robots:
            if rob.pos not in count:
                count[rob.pos] = 0
            count[rob.pos] += 1
        return count

    def advance_sec(self):
        for rob in self.robots:
            future = list(point_op(add, rob.pos, rob.vel))
            if future[0] < 0:
                future[0] += self.w
            if future[0] >= self.w:
                future[0] -= self.w
            if future[1] < 0:
                future[1] += self.h
            if future[1] >= self.h:
                future[1] -= self.h
            rob.pos = tuple(future)

    def safety_factor(self) -> int:
        topleft     = range(0, floor(self.w / 2)),          range(0, floor(self.h / 2))
        topright    = range(floor(self.w / 2) + 1, self.w), range(0, floor(self.h / 2))
        bottomleft  = range(0, floor(self.w / 2)),          range(floor(self.h / 2) + 1, self.h)
        bottomright = range(floor(self.w / 2) + 1, self.w), range(floor(self.h / 2) + 1, self.h)
        quadrant_counts: list[int] = [0, 0, 0, 0]
        for n, quadrange in enumerate((topleft, topright, bottomleft, bottomright)):
            for rob in self.robots:
                if rob.pos[0] in quadrange[0] and rob.pos[1] in quadrange[1]:
                    quadrant_counts[n] += 1
        return reduce(mul, quadrant_counts)

In [29]:
rm = RobotRoom(puzzle, *rmsize_puzzle)

In [30]:
for _ in range(100):
    rm.advance_sec()

In [31]:
rm.safety_factor()

224554908